In [1]:
import numpy as np
import pandas as pd
import autokeras as ak
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# Загрузка датасета Iris
iris = load_iris()
# Разделение данных на обучающий и тестовый наборы
x_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.75, test_size=0.25)

print(x_train[:10])

Using TensorFlow backend
[[5.1 3.8 1.9 0.4]
 [7.1 3.  5.9 2.1]
 [6.1 3.  4.6 1.4]
 [6.1 2.8 4.  1.3]
 [7.6 3.  6.6 2.1]
 [4.5 2.3 1.3 0.3]
 [5.6 3.  4.1 1.3]
 [5.7 2.5 5.  2. ]
 [7.2 3.6 6.1 2.5]
 [7.7 3.  6.1 2.3]]


In [2]:
# Применение PCA на обучающем наборе
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(x_train)
print(X_train_pca[:10])

[[-2.26558136  0.41512238]
 [ 2.56186091  0.40042054]
 [ 0.84239656 -0.00394666]
 [ 0.31002822 -0.04701114]
 [ 3.3397646   0.62405015]
 [-2.88319513 -0.96982988]
 [ 0.20084331 -0.25041485]
 [ 1.30812607 -0.74522231]
 [ 2.85366596  0.83737358]
 [ 3.01622417  0.75188623]]


In [3]:
# Создание нового датасета с использованием новых признаков
new_data = np.concatenate((X_train_pca, y_train.reshape(-1, 1)), axis=1)
new_data = pd.DataFrame(new_data, columns=['feature1', 'feature2', 'target'])

In [4]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=10
)  # It tries 10 different models.

In [5]:
# Feed the structured data classifier with training data.
clf.fit(
    X_train_pca,
    y_train,
    epochs=50,
)

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.5625

Best val_accuracy So Far: 0.875
Total elapsed time: 00h 00m 24s


Epoch 1/50
4/4 [==============================] - 0s 1ms/step - loss: 1.0687 - accuracy: 0.4821
Epoch 2/50
4/4 [==============================] - 0s 1ms/step - loss: 1.0327 - accuracy: 0.6339
Epoch 3/50
4/4 [==============================] - 0s 1ms/step - loss: 0.9971 - accuracy: 0.7321
Epoch 4/50
4/4 [==============================] - 0s 1ms/step - loss: 0.9592 - accuracy: 0.7768
Epoch 5/50
4/4 [==============================] - 0s 1ms/step - loss: 0.9183 - accuracy: 0.7857
Epoch 6/50
4/4 [==============================] - 0s 1ms/step - loss: 0.8745 - accuracy: 0.7857
Epoch 7/50
4/4 [==============================] - 0s 1ms/step - loss: 0.8275 - accuracy: 0.7946
Epoch 8/50
4/4 [==============================] - 0s 1ms/step - loss: 0.7783 - accuracy: 0.7946
Epoch 9/50
4/4 [==============================] - 0s 1ms/step - loss: 0.7284 - accuracy: 0.8125
Epoch 10/50
4/4 [==============================] - 0s 1ms/step - loss: 0.6793 - accuracy: 0.8125
Epoch 11/50
4/4 [======================

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [10]:
# Прогнозирование на новых данных
X_test_pca = pca.transform(X_test)
predict_res = clf.predict(X_test_pca)
print(predict_res)

2/2 [==============================] - 0s 1ms/step
[['2']
 ['2']
 ['0']
 ['1']
 ['0']
 ['2']
 ['2']
 ['1']
 ['0']
 ['0']
 ['2']
 ['0']
 ['0']
 ['0']
 ['1']
 ['1']
 ['2']
 ['1']
 ['2']
 ['0']
 ['1']
 ['1']
 ['2']
 ['2']
 ['2']
 ['2']
 ['0']
 ['0']
 ['0']
 ['1']
 ['0']
 ['1']
 ['1']
 ['2']
 ['2']
 ['0']
 ['0']
 ['1']]


In [11]:
# Evaluate the best model with testing data.
eval_res = clf.evaluate(X_test_pca, y_test)
print(eval_res)

2/2 [==============================] - 0s 2ms/step - loss: 0.1433 - accuracy: 1.0000
[0.14327585697174072, 1.0]


In [12]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 multi_category_encoding (M  (None, 2)                 0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 2)                 5         
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                96        
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  